In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import BallTree
from sklearn.metrics import mean_absolute_error
from joblib import Parallel, delayed

# 데이터 로드
merged_data = pd.read_csv('merged_data_cleaned.csv').reset_index(drop=True)

# Train과 Test 데이터 분리(_type으로 분리가 안 되서 deposit이 0인가 아닌가로 분리함)
train_df = merged_data[merged_data['deposit'] != 0].copy()
test_df = merged_data[merged_data['deposit'] == 0].copy()

# 학습이 너무 오래걸려서 어쩔 수 없이 특성 선택
features = ['latitude', 'longitude', 'area_m2', 'floor', 'built_year', 'contract_year_month']

# Train 데이터를 train과 validation으로 분리
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# 스케일링
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df[features])
val_scaled = scaler.transform(val_df[features])
test_scaled = scaler.transform(test_df[features])

# 특성 가중치 조정
# 그리드 서치로 최적 값 찾음 Valid MAE: 4646.19  
weights = np.array([10, 10, 10, 0.1, 10, 2])  # latitude, longitude, area_m2, floor, built_year, contract_year_month 순서
train_scaled = train_scaled * weights
val_scaled = val_scaled * weights
test_scaled = test_scaled * weights

# BallTree 구축 (맨해튼 거리 사용) - 맨해튼이 유클리드 보다 학습이 오래 걸리긴 한데 성능 더 잘나옴 
tree = BallTree(train_scaled, leaf_size=40, metric='manhattan')

# 예측 함수 (거리에 따른 가중치 함수 조정)
def predict_deposit(neighbors, distances):
    weights = np.exp(-distances)  # 지수 함수 사용
    weighted_deposits = neighbors['deposit'] * weights
    return np.sum(weighted_deposits) / np.sum(weights)

# 단일 예측 함수
def predict_single(data_point, k):
    distances, indices = tree.query([data_point], k=k)
    neighbors = train_df.iloc[indices[0]]
    return predict_deposit(neighbors, distances[0])

# 병렬 예측 수행 함수
def make_predictions_parallel(data_scaled, k):
    return Parallel(n_jobs=-1)(delayed(predict_single)(data_point, k) for data_point in data_scaled)

# 앙상블 예측 함수
def ensemble_predictions(data_scaled, k_values):
    all_predictions = Parallel(n_jobs=-1)(
        delayed(make_predictions_parallel)(data_scaled, k) for k in k_values
    )
    return np.mean(all_predictions, axis=0)

# k 값 리스트 정의
k_values = [3, 5, 7, 9]

# Validation 예측
val_predictions = ensemble_predictions(val_scaled, k_values)

# Validation MAE 계산
val_mae = mean_absolute_error(val_df['deposit'], val_predictions)
print(f"Validation MAE: {val_mae}")

# Test 예측
test_predictions = ensemble_predictions(test_scaled, k_values)

# 결과 저장
submission_df = pd.DataFrame({
    'index': range(len(test_predictions)),
    'deposit': test_predictions
})

submission_df.to_csv('submission.csv', index=False)

print("예측이 완료되었습니다. 'submission.csv' 파일을 확인하세요.")

Validation MAE: 4646.197436650035
예측이 완료되었습니다. 'submission.csv' 파일을 확인하세요.
